In [1]:
from __future__ import annotations

import pathlib
from typing import Optional, Any

import transformers
import wandb
import torch
from torch.utils.data import DataLoader
import typer
import yaml
import torchdata.datapipes as dp
import json
import time

import audiocap.metrics
import audiocap.data
import audiocap.callbacks
import audiocap.models

from train_whisper_supervised import get_whisper_model


In [2]:
import os
assert torch.cuda.is_available()
!nvidia-smi

Wed May  3 16:13:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   36C    P0    59W / 400W |      2MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:46:00.0 Off |                    0 |
| N/A   

In [3]:
torch.cuda.device_count()

5

In [4]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_properties(i))

DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":50, please report a bug to PyTorch. 

CUDA call was originally invoked at:

['  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 194, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 87, in _run_code\n    exec(code, run_globals)\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>\n    app.launch_new_instance()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance\n    app.start()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start\n    self.io_loop.start(

In [5]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
torch.tensor([1,2,3,], device="cuda:0")

DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":50, please report a bug to PyTorch. 

CUDA call was originally invoked at:

['  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 194, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 87, in _run_code\n    exec(code, run_globals)\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>\n    app.launch_new_instance()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance\n    app.start()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start\n    self.io_loop.start(

In [68]:
load_checkpoint = pathlib.Path("../checkpoints/stoic-totem-29/checkpoint-18900")
dataset_dir = pathlib.Path("../../maratmp/audio-captioning/data/clotho_v2.1/audiofolder/")
output_dir = pathlib.Path("../inference_outputs")
generate_config = pathlib.Path("../configs/generate_clotho_test.yaml")


In [69]:
with open(generate_config, "r") as f:
    generate_config_dict: dict = yaml.safe_load(f)

In [70]:
generate_args_dict = generate_config_dict["generate_args"]
architecture_config = generate_config_dict["architecture"]
architecture_name = architecture_config["name"]
use_pretrained_encoder = architecture_config["use_pretrained_whisper_encoder"]
use_pretrained_decoder = architecture_config["use_pretrained_whisper_decoder"]

only_allowed_tokens = generate_config_dict["only_allowed_tokens"]
batch_size = generate_config_dict["batch_size"]

data_config = generate_config_dict["data"]
dataset_name = data_config["dataset_name"]
task = data_config["task"]
dataset_type = data_config["dataset_type"]
data_limit = data_config["data_limit"]
run_name = load_checkpoint.parent.name  

print("run name: ", run_name)

run name:  stoic-totem-29


In [9]:
config = transformers.WhisperConfig.from_pretrained(architecture_name)
tokenizer = transformers.WhisperTokenizer.from_pretrained(architecture_name, language="en", task="transcribe")
feature_extractor = transformers.WhisperFeatureExtractor.from_pretrained(architecture_name)
collator = audiocap.data.DataCollatorAudioSeq2SeqWithPadding(tokenizer, feature_extractor)
assert isinstance(config, transformers.WhisperConfig)
model = get_whisper_model(architecture_name, config, load_checkpoint, use_pretrained_encoder, use_pretrained_decoder)

tuned_params = sum(p.shape.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.shape.numel() for p in model.parameters())
print(f"Number of trained parameters: {tuned_params}/{total_params} = {tuned_params/total_params*100:.2f}%")

Number of trained parameters: 1543304960/1543304960 = 100.00%


In [65]:
model.to(device)

DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":50, please report a bug to PyTorch. 

CUDA call was originally invoked at:

['  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 194, in _run_module_as_main\n    return _run_code(code, main_globals, None,\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/runpy.py", line 87, in _run_code\n    exec(code, run_globals)\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>\n    app.launch_new_instance()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance\n    app.start()\n', '  File "/home/xhajek9/miniconda3/envs/malach23/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start\n    self.io_loop.start(

In [71]:
# set eval dataset
audioset_dir = None
audiocaps_dir = None
clotho_dir = None

if dataset_name == "clotho":
    ds = audiocap.data.load_clotho(dataset_dir, tokenizer, feature_extractor, None, 0, 0, 0)
elif dataset_name == "audioset":
    ds = audiocap.data.load_audioset(dataset_dir, tokenizer, feature_extractor, None, 0, 0, 0)
elif dataset_name == "audiocaps":
    ds = audiocap.data.load_audiocaps(dataset_dir, tokenizer, feature_extractor, None, 0, 0, 0)
else:
    raise ValueError(f"Unknown dataset name: {dataset_name}")

In [72]:
# generate captions (no filename)
dataloader = DataLoader(ds[dataset_type].pipe, batch_size=1, collate_fn=collator, num_workers=4)
all_predictions = []
for i, batch in enumerate(dataloader):
    if data_limit and i >= data_limit/batch_size:
        break

    prediction_ids = model.generate(batch["input_features"], 
                                    forced_ac_decoder_ids=batch["forced_ac_decoder_ids"], 
                                    **generate_args_dict)
    predictions = tokenizer.batch_decode(prediction_ids, skip_special_tokens=False)
    all_predictions.extend([{"caption": p} for p in enumerate(predictions)])
    print(predictions)

['<|startoftranscript|><|en|><|transcribe|><|notimestamps|>clotho > caption: Music plays nearby as a person sprays on a surface<|endoftext|>']
['<|startoftranscript|><|en|><|transcribe|><|notimestamps|>clotho > caption: A large motor vehicle engine is running and vibrating<|endoftext|>']


In [74]:
all_predictions

[{'caption': (0,
   '<|startoftranscript|><|en|><|transcribe|><|notimestamps|>clotho > caption: Music plays nearby as a person sprays on a surface<|endoftext|>')},
 {'caption': (0,
   '<|startoftranscript|><|en|><|transcribe|><|notimestamps|>clotho > caption: A large motor vehicle engine is running and vibrating<|endoftext|>')}]

In [ ]:
# generate captions (filename, but bs=1)

In [61]:
# save predictions as jsonl to output_dir
log_dir = output_dir / "logs"
output_dir.mkdir(parents=True, exist_ok=True)
log_dir.mkdir(parents=True, exist_ok=True)
timestamp = round(time.time())
with open(output_dir / f"{run_name}_{dataset_name}_{dataset_type}_{timestamp}.jsonl", "w") as f:
    for prediction in all_predictions:
        f.write(json.dumps(prediction) + "\n")

# copy generate_config to log_dir
generate_config_dst = log_dir / f"{timestamp}.yaml"
generate_config_dst.write_text(generate_config.read_text())

332

In [32]:
tokenizer.decode(prediction[0])

'<|startoftranscript|><|en|><|transcribe|><|notimestamps|>clotho > caption: A motor vehicle engine is running and vibrating, and people are talking in the background<|endoftext|>'

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

NameError: name 'device' is not defined